# Massive Label Grid + On-the-Fly Model Grid Search

This notebook runs a large grid search over labeling parameters and, for each label setup, a small model hyperparameter grid.

Primary objective: `move_weighted_acc` on the test split.

`move_weighted_acc` measures closeness to true labels with sample weights based on absolute future move `|Y|`. Large moves dominate; small moves have limited impact.

In [ ]:
import sys
import io
import time
import itertools
import contextlib
from pathlib import Path
from typing import Dict, List, Any

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "RESEARCH").exists():
    for parent in PROJECT_ROOT.parents:
        if (parent / "RESEARCH").exists():
            PROJECT_ROOT = parent
            break

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from RESEARCH.config import cfg
from RESEARCH.data_loader import load_market_data
from RESEARCH.labeling import create_balanced_labels, gaussian_smooth_centered
from RESEARCH.astro_engine import (
    init_ephemeris,
    calculate_bodies_for_dates_multi,
    calculate_aspects_from_cache,
    calculate_phases_for_dates,
    precompute_angles_for_dates,
)
from RESEARCH.features import build_full_features
from RESEARCH.model_training import split_dataset, prepare_xy, train_xgb_model, calc_metrics, check_cuda_available

pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

print(f"Python: {sys.version.split()[0]}")
print(f"PROJECT_ROOT: {PROJECT_ROOT}")

In [ ]:
# Search configuration
DATA_START = "2017-11-01"
RUN_TAG = "massive_label_weighted"

# Giant labeling grid
LABEL_GRID = {
    "horizon": [1, 2, 3],
    "move_share": [0.35, 0.50, 0.65],
    "gauss_window": [51, 101, 151, 201, 251, 301, 401],
    "gauss_std": [15.0, 25.0, 35.0, 50.0, 70.0, 90.0, 110.0],
    "price_mode": ["raw", "log"],
    "label_mode": ["balanced_detrended", "balanced_future_return"],
}

# Astro feature grid
FEATURE_GRID = {
    "coord_mode": ["geo", "helio", "both"],
    "orb_multiplier": [0.10, 0.15, 0.25, 0.50, 1.00],
    "include_phases": [True],
}

# Small model grid (on-the-fly per label combo)
MODEL_GRID = {
    "n_estimators": [300, 500],
    "max_depth": [3, 5],
    "learning_rate": [0.03, 0.05],
    "subsample": [0.8],
    "colsample_bytree": [0.8],
    "weight_power": [1.0, 1.5],
}

# Metric settings: stronger focus on large |Y|
MOVE_WEIGHT_POWER = 1.5
MOVE_WEIGHT_CLIP_Q = 0.98
TOP_MOVE_QUANTILE = 0.80

# Runtime and checkpoint controls
SEED = 42
SHUFFLE_LABEL_COMBOS = True
MAX_LABEL_COMBOS = None
MAX_TOTAL_MODEL_EVALS = None
CHECKPOINT_EVERY = 25
MIN_DATASET_ROWS = 300
RESUME_FROM_CHECKPOINT = True

# Quick dry-run mode
TEST_MODE = False
if TEST_MODE:
    LABEL_GRID = {
        "horizon": [1],
        "move_share": [0.50],
        "gauss_window": [201],
        "gauss_std": [50.0],
        "price_mode": ["raw"],
        "label_mode": ["balanced_detrended"],
    }
    FEATURE_GRID = {
        "coord_mode": ["both"],
        "orb_multiplier": [0.15],
        "include_phases": [True],
    }
    MODEL_GRID = {
        "n_estimators": [300],
        "max_depth": [4],
        "learning_rate": [0.03],
        "subsample": [0.8],
        "colsample_bytree": [0.8],
        "weight_power": [1.0, 1.5],
    }
    MAX_LABEL_COMBOS = 4
    RUN_TAG = "massive_label_weighted_test"

print("Configured. TEST_MODE=", TEST_MODE)

In [ ]:
def generate_grid(grid: Dict[str, List[Any]]) -> List[Dict[str, Any]]:
    keys = list(grid.keys())
    values = [grid[k] for k in keys]
    return [dict(zip(keys, combo)) for combo in itertools.product(*values)]


def pair_signature(feature_combo: Dict[str, Any], label_combo: Dict[str, Any]) -> str:
    return str((tuple(sorted(feature_combo.items())), tuple(sorted(label_combo.items()))))


def move_weights(y_move: np.ndarray, power: float = MOVE_WEIGHT_POWER, clip_q: float = MOVE_WEIGHT_CLIP_Q) -> np.ndarray:
    abs_move = np.abs(np.asarray(y_move, dtype=np.float64))
    if len(abs_move) == 0:
        return np.array([], dtype=np.float64)
    cap = float(np.quantile(abs_move, clip_q))
    if cap <= 0.0:
        cap = float(abs_move.max()) if abs_move.max() > 0 else 1.0
    w = np.clip(abs_move / cap, 0.0, 1.0) ** power
    return np.maximum(w, 1e-8)


def move_weighted_acc(y_true: np.ndarray, y_pred: np.ndarray, y_move: np.ndarray) -> float:
    w = move_weights(y_move)
    ok = (y_true == y_pred).astype(np.float64)
    return float(np.sum(w * ok) / np.sum(w))


def move_direction_capture(y_pred: np.ndarray, y_move: np.ndarray) -> float:
    w = move_weights(y_move)
    pred_sign = np.where(y_pred == 1, 1.0, -1.0)
    true_sign = np.where(np.asarray(y_move, dtype=np.float64) >= 0, 1.0, -1.0)
    signed = pred_sign * true_sign
    return float(np.sum(w * signed) / np.sum(w))


def top_move_acc(y_true: np.ndarray, y_pred: np.ndarray, y_move: np.ndarray, q: float = TOP_MOVE_QUANTILE) -> float:
    abs_move = np.abs(np.asarray(y_move, dtype=np.float64))
    thr = float(np.quantile(abs_move, q))
    mask = abs_move >= thr
    if mask.sum() == 0:
        return 0.0
    return float((y_true[mask] == y_pred[mask]).mean())


def model_predict_binary(model, X: np.ndarray, threshold: float) -> np.ndarray:
    constant_class = getattr(model, "constant_class", None)
    if constant_class is not None:
        return np.full(X.shape[0], int(constant_class), dtype=np.int32)
    X_scaled = model.scaler.transform(X)
    proba = model.model.predict_proba(X_scaled)[:, 1]
    return (proba >= threshold).astype(np.int32)


def tune_threshold_by_move_weighted_score(model, X_val, y_val, y_move_val):
    constant_class = getattr(model, "constant_class", None)
    if constant_class is not None:
        pred = np.full(X_val.shape[0], int(constant_class), dtype=np.int32)
        return 0.5, move_weighted_acc(y_val, pred, y_move_val)

    X_scaled = model.scaler.transform(X_val)
    proba = model.model.predict_proba(X_scaled)[:, 1]

    best_t = 0.5
    best_score = -1.0
    best_recall_min = -1.0

    for t in np.linspace(0.05, 0.95, 91):
        pred = (proba >= t).astype(np.int32)
        score = move_weighted_acc(y_val, pred, y_move_val)
        m = calc_metrics(y_val, pred, [0, 1])

        if score > best_score:
            best_t = float(t)
            best_score = float(score)
            best_recall_min = float(m["recall_min"])
        elif np.isclose(score, best_score) and m["recall_min"] > best_recall_min:
            best_t = float(t)
            best_recall_min = float(m["recall_min"])

    return best_t, best_score


def split_dataset_silent(df: pd.DataFrame):
    with contextlib.redirect_stdout(io.StringIO()):
        return split_dataset(df)


def prefix_non_date_columns(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    if df is None or df.empty:
        return pd.DataFrame()
    rename_map = {c: f"{prefix}{c}" for c in df.columns if c != "date"}
    return df.rename(columns=rename_map)


def build_label_frame(df_market: pd.DataFrame, combo: Dict[str, Any]) -> pd.DataFrame:
    horizon = int(combo["horizon"])
    price_mode = str(combo["price_mode"])
    label_mode = str(combo["label_mode"])

    df_labels = create_balanced_labels(
        df_market=df_market,
        horizon=horizon,
        move_share=float(combo["move_share"]),
        gauss_window=int(combo["gauss_window"]),
        gauss_std=float(combo["gauss_std"]),
        price_mode=price_mode,
        label_mode=label_mode,
        verbose=False,
    )

    close = df_market["close"].astype(float)
    raw_move = close.shift(-horizon) - close

    if price_mode == "log":
        base = np.log(close)
    else:
        base = close

    if label_mode == "balanced_detrended":
        smooth = gaussian_smooth_centered(base, int(combo["gauss_window"]), float(combo["gauss_std"]))
        detrended = base - smooth
        label_space_move = detrended.shift(-horizon) - detrended
    else:
        label_space_move = base.shift(-horizon) - base

    df_y = pd.DataFrame({
        "date": pd.to_datetime(df_market["date"]),
        "y_move_raw": raw_move.to_numpy(dtype=np.float64),
        "y_move_label": label_space_move.to_numpy(dtype=np.float64),
    })

    df_labels = df_labels.copy()
    df_labels["date"] = pd.to_datetime(df_labels["date"])
    out = df_labels.merge(df_y, on="date", how="left")
    out = out.dropna(subset=["y_move_raw", "y_move_label"]).sort_values("date").reset_index(drop=True)
    return out


def prepare_astro_state(df_market: pd.DataFrame, settings, coord_mode: str, include_phases: bool = True) -> Dict[str, Any]:
    df_bodies, geo_by_date, helio_by_date = calculate_bodies_for_dates_multi(
        df_market["date"], settings, coord_mode=coord_mode, progress=True
    )
    df_bodies = df_bodies.copy()
    df_bodies["date"] = pd.to_datetime(df_bodies["date"])

    state = {"df_bodies": df_bodies, "angles": {}, "df_phases": None}

    if coord_mode in {"geo", "both"} and geo_by_date:
        state["angles"]["geo"] = precompute_angles_for_dates(geo_by_date, progress=True)
    if coord_mode in {"helio", "both"} and helio_by_date:
        state["angles"]["helio"] = precompute_angles_for_dates(helio_by_date, progress=True)

    if include_phases:
        if coord_mode == "geo":
            df_phases = calculate_phases_for_dates(geo_by_date, progress=False)
            df_phases["date"] = pd.to_datetime(df_phases["date"])
            state["df_phases"] = df_phases
        elif coord_mode == "helio":
            df_phases = calculate_phases_for_dates(helio_by_date, progress=False)
            df_phases = prefix_non_date_columns(df_phases, "helio_")
            df_phases["date"] = pd.to_datetime(df_phases["date"])
            state["df_phases"] = df_phases
        else:
            df_geo = prefix_non_date_columns(calculate_phases_for_dates(geo_by_date, progress=False), "geo_")
            df_hel = prefix_non_date_columns(calculate_phases_for_dates(helio_by_date, progress=False), "helio_")
            df_geo["date"] = pd.to_datetime(df_geo["date"])
            df_hel["date"] = pd.to_datetime(df_hel["date"])
            state["df_phases"] = df_geo.merge(df_hel, on="date", how="outer")

    return state


def build_aspects_for_mode(astro_state: Dict[str, Any], settings, coord_mode: str, orb_mult: float) -> pd.DataFrame:
    if coord_mode == "geo":
        return calculate_aspects_from_cache(astro_state["angles"]["geo"], settings, orb_mult=orb_mult, prefix="", progress=False)
    if coord_mode == "helio":
        return calculate_aspects_from_cache(astro_state["angles"]["helio"], settings, orb_mult=orb_mult, prefix="helio_", progress=False)

    df_geo = calculate_aspects_from_cache(astro_state["angles"]["geo"], settings, orb_mult=orb_mult, prefix="geo_", progress=False)
    df_hel = calculate_aspects_from_cache(astro_state["angles"]["helio"], settings, orb_mult=orb_mult, prefix="helio_", progress=False)
    return pd.concat([df_geo, df_hel], ignore_index=True)


def is_better(candidate: Dict[str, Any], current: Dict[str, Any]) -> bool:
    if candidate is None:
        return False
    if current is None:
        return True

    c1 = float(candidate.get("test_move_weighted_acc", np.nan))
    c2 = float(current.get("test_move_weighted_acc", np.nan))

    if np.isnan(c1):
        return False
    if np.isnan(c2):
        return True

    if c1 > c2:
        return True
    if np.isclose(c1, c2) and float(candidate.get("recall_min", -1)) > float(current.get("recall_min", -1)):
        return True
    if np.isclose(c1, c2) and np.isclose(float(candidate.get("recall_min", -1)), float(current.get("recall_min", -1))):
        return float(candidate.get("mcc", -1)) > float(current.get("mcc", -1))
    return False

In [ ]:
rng = np.random.default_rng(SEED)

label_combos = generate_grid(LABEL_GRID)
feature_combos = generate_grid(FEATURE_GRID)
model_combos = generate_grid(MODEL_GRID)

if SHUFFLE_LABEL_COMBOS:
    rng.shuffle(label_combos)
if MAX_LABEL_COMBOS is not None:
    label_combos = label_combos[: int(MAX_LABEL_COMBOS)]

print(f"Label combos:   {len(label_combos)}")
print(f"Feature combos: {len(feature_combos)}")
print(f"Model combos:   {len(model_combos)}")
print(f"Total model evals (upper bound): {len(label_combos) * len(feature_combos) * len(model_combos)}")

df_market = load_market_data()
df_market = df_market[df_market["date"] >= DATA_START].copy()
df_market["date"] = pd.to_datetime(df_market["date"])
df_market = df_market.sort_values("date").reset_index(drop=True)

print(f"Market rows: {len(df_market)}")
print(f"Date range: {df_market['date'].min().date()} -> {df_market['date'].max().date()}")

settings = init_ephemeris()
_, device = check_cuda_available()
print(f"Training device: {device}")

# Precompute astro data per coordinate mode
astro_by_mode: Dict[str, Dict[str, Any]] = {}
coord_modes_needed = sorted(set(c["coord_mode"] for c in feature_combos))

for mode in coord_modes_needed:
    print(f"\n[ASTRO PREP] mode={mode}")
    astro_by_mode[mode] = prepare_astro_state(df_market, settings, mode, include_phases=True)

label_cache: Dict[str, pd.DataFrame] = {}
feature_cache: Dict[str, pd.DataFrame] = {}
results: List[Dict[str, Any]] = []
best_global = None
eval_counter = 0
pair_counter = 0
started_at = time.time()

reports_dir = cfg.reports_dir
reports_dir.mkdir(parents=True, exist_ok=True)
checkpoint_path = reports_dir / f"{RUN_TAG}_checkpoint.csv"
done_pairs = set()

if RESUME_FROM_CHECKPOINT and checkpoint_path.exists():
    df_prev = pd.read_csv(checkpoint_path)
    if "pair_key" in df_prev.columns:
        results = df_prev.to_dict(orient="records")
        done_pairs = set(df_prev["pair_key"].astype(str).tolist())
        if not df_prev.empty:
            best_global = df_prev.sort_values(["test_move_weighted_acc", "recall_min", "mcc"], ascending=[False, False, False]).iloc[0].to_dict()
        print(f"[RESUME] loaded rows={len(df_prev)}, done_pairs={len(done_pairs)} from {checkpoint_path}")

for f_idx, f_combo in enumerate(feature_combos, start=1):
    f_key = str(tuple(sorted(f_combo.items())))

    if f_key not in feature_cache:
        state = astro_by_mode[f_combo["coord_mode"]]
        df_aspects = build_aspects_for_mode(
            astro_state=state,
            settings=settings,
            coord_mode=f_combo["coord_mode"],
            orb_mult=float(f_combo["orb_multiplier"]),
        )
        df_features = build_full_features(
            state["df_bodies"],
            df_aspects,
            df_phases=state["df_phases"] if bool(f_combo["include_phases"]) else None,
        )
        df_features = df_features.copy()
        df_features["date"] = pd.to_datetime(df_features["date"])
        feature_cache[f_key] = df_features

    df_features = feature_cache[f_key]
    print(f"\n[FEATURE {f_idx}/{len(feature_combos)}] {f_combo}")

    for l_idx, l_combo in enumerate(label_combos, start=1):
        sig = pair_signature(f_combo, l_combo)
        if sig in done_pairs:
            continue

        if MAX_TOTAL_MODEL_EVALS is not None and eval_counter >= int(MAX_TOTAL_MODEL_EVALS):
            break

        l_key = str(tuple(sorted(l_combo.items())))
        if l_key not in label_cache:
            label_cache[l_key] = build_label_frame(df_market, l_combo)

        df_labels = label_cache[l_key]
        df_dataset = df_features.merge(df_labels[["date", "target", "y_move_raw", "y_move_label"]], on="date", how="inner")
        df_dataset = df_dataset.sort_values("date").drop_duplicates(subset=["date"]).reset_index(drop=True)

        if len(df_dataset) < MIN_DATASET_ROWS:
            done_pairs.add(sig)
            continue

        train_df, val_df, test_df = split_dataset_silent(df_dataset)
        if len(train_df) < 100 or len(val_df) < 30 or len(test_df) < 30:
            done_pairs.add(sig)
            continue

        feature_cols = [c for c in df_dataset.columns if c not in ["date", "target", "y_move_raw", "y_move_label"]]
        X_train, y_train = prepare_xy(train_df, feature_cols)
        X_val, y_val = prepare_xy(val_df, feature_cols)
        X_test, y_test = prepare_xy(test_df, feature_cols)
        y_move_val = val_df["y_move_raw"].to_numpy(dtype=np.float64)
        y_move_test = test_df["y_move_raw"].to_numpy(dtype=np.float64)

        best_local = None

        for m_combo in model_combos:
            if MAX_TOTAL_MODEL_EVALS is not None and eval_counter >= int(MAX_TOTAL_MODEL_EVALS):
                break

            eval_counter += 1

            try:
                model = train_xgb_model(
                    X_train, y_train,
                    X_val, y_val,
                    feature_names=feature_cols,
                    n_classes=2,
                    device=device,
                    verbose=False,
                    **m_combo,
                )

                best_t, val_score = tune_threshold_by_move_weighted_score(model, X_val, y_val, y_move_val)
                y_pred_test = model_predict_binary(model, X_test, threshold=best_t)

                core = calc_metrics(y_test, y_pred_test, [0, 1])
                row = {
                    "pair_key": sig,
                    **{f"label_{k}": v for k, v in l_combo.items()},
                    **{f"feature_{k}": v for k, v in f_combo.items()},
                    **{f"model_{k}": v for k, v in m_combo.items()},
                    "threshold": float(best_t),
                    "val_move_weighted_acc": float(val_score),
                    "test_move_weighted_acc": move_weighted_acc(y_test, y_pred_test, y_move_test),
                    "test_direction_capture": move_direction_capture(y_pred_test, y_move_test),
                    "test_top_move_acc": top_move_acc(y_test, y_pred_test, y_move_test, q=TOP_MOVE_QUANTILE),
                    "acc": float(core["acc"]),
                    "bal_acc": float(core["bal_acc"]),
                    "mcc": float(core["mcc"]),
                    "f1_macro": float(core["f1_macro"]),
                    "recall_min": float(core["recall_min"]),
                    "recall_gap": float(core["recall_gap"]),
                    "n_train": int(len(train_df)),
                    "n_val": int(len(val_df)),
                    "n_test": int(len(test_df)),
                    "eval_id": int(eval_counter),
                    "elapsed_min": float((time.time() - started_at) / 60.0),
                }

                if is_better(row, best_local):
                    best_local = row

            except Exception as exc:
                print(f"[WARN] skipped model combo due to error: {exc}")
                continue

        done_pairs.add(sig)
        pair_counter += 1

        if best_local is not None:
            results.append(best_local)
            if is_better(best_local, best_global):
                best_global = best_local
                print(
                    f"[NEW BEST] eval={best_global['eval_id']} "
                    f"test_move_weighted_acc={best_global['test_move_weighted_acc']:.4f} "
                    f"feature={f_combo} label_h={l_combo['horizon']} gw={l_combo['gauss_window']} gs={l_combo['gauss_std']}"
                )

        if pair_counter % CHECKPOINT_EVERY == 0 and len(results) > 0:
            df_ckpt = pd.DataFrame(results).sort_values(
                ["test_move_weighted_acc", "recall_min", "mcc"],
                ascending=[False, False, False],
            )
            df_ckpt.to_csv(checkpoint_path, index=False)
            print(f"[CHECKPOINT] {checkpoint_path} rows={len(df_ckpt)} done_pairs={len(done_pairs)}")

    if MAX_TOTAL_MODEL_EVALS is not None and eval_counter >= int(MAX_TOTAL_MODEL_EVALS):
        break

print(f"\nFinished. model_evals={eval_counter}, kept_rows={len(results)}, done_pairs={len(done_pairs)}")

if results:
    df_results = pd.DataFrame(results).sort_values(
        ["test_move_weighted_acc", "recall_min", "mcc"],
        ascending=[False, False, False],
    ).reset_index(drop=True)
    df_results.to_csv(checkpoint_path, index=False)
    print(f"Saved final results: {checkpoint_path}")
else:
    df_results = pd.DataFrame()
    print("No valid combinations evaluated.")

In [ ]:
if df_results.empty:
    print("No results to display.")
else:
    print("Top 20 by test_move_weighted_acc:")
    display_cols = [
        "test_move_weighted_acc", "val_move_weighted_acc", "test_top_move_acc", "test_direction_capture",
        "recall_min", "mcc", "feature_coord_mode", "feature_orb_multiplier",
        "label_horizon", "label_move_share", "label_gauss_window", "label_gauss_std",
        "label_price_mode", "label_label_mode",
        "model_n_estimators", "model_max_depth", "model_learning_rate", "model_weight_power",
        "threshold",
    ]
    display(df_results[display_cols].head(20))

    plt.figure(figsize=(8, 5))
    plt.scatter(df_results["test_move_weighted_acc"], df_results["recall_min"], alpha=0.5)
    plt.xlabel("test_move_weighted_acc")
    plt.ylabel("recall_min")
    plt.title("Quality Frontier: large-move objective vs balanced recall")
    plt.grid(alpha=0.25)
    plt.show()

    best = df_results.iloc[0].to_dict()
    print("Best objective row:")
    for k in [
        "test_move_weighted_acc", "test_top_move_acc", "test_direction_capture",
        "recall_min", "mcc", "feature_coord_mode", "feature_orb_multiplier",
        "label_horizon", "label_move_share", "label_gauss_window", "label_gauss_std",
        "label_price_mode", "label_label_mode",
        "model_n_estimators", "model_max_depth", "model_learning_rate", "model_weight_power",
        "threshold",
    ]:
        print(f"  {k}: {best[k]}")